In [ ]:
%pip install selenium


## Find the Games Played Today

In [ ]:
import csv
from datetime import date, timedelta

def get_games_today(filename):
    """
    Returns a list of tuples representing the games being played today from a CSV file.
    Each tuple contains the date, time, and two teams playing against each other.
    """
    games_today = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        tomorrow = date.today() + timedelta(days=0)
        for row in reader:
            game_date = row[0]
            game_time = row[1]
            team1 = row[2]
            team2 = row[3]
            if game_date == str(tomorrow):
                games_today.append((game_date, game_time, team1, team2))
    return games_today

In [ ]:
games_today = get_games_today('2022_2023_NHL_Schedule.csv')
print(games_today)

## Unsucessful Scraping Attempt

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Set up the Chrome driver with the path to your chromedriver executable

driver = webdriver.Chrome()

url = "https://www.nhl.com/stats/teams"

# Load the NHL stats/teams page with Selenium
driver.get(url)

# Wait for the table element to become available before parsing it with BeautifulSoup
try:
    table_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//table[@class='rt-table']"))
    )
    soup = BeautifulSoup(driver.page_source, "html.parser")
    table = soup.find("table", {"class": "rt-table"})

    # Get the table header row and extract the column names
    header_row = table.find("thead").find("tr")
    column_names = [th.text.strip() for th in header_row.find_all("th")]

    # Create an empty list to store the table data rows
    rows = []

    # Iterate through the table body rows and extract the data
    for tr in table.find("tbody").find_all("tr"):
        # Extract the column data for each row
        row = [td.text.strip() for td in tr.find_all("td")]
        # Add the row to the list of rows
        rows.append(row)

    # Print the column names and data rows
    print(column_names)
    for row in rows:
        print(row)

finally:
    # Close the browser window
    driver.quit()



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Replace the URL and driver path with your own values
url = "https://www.hockey-reference.com/leagues/NHL_2023_games.html"


# Create a new instance of the Firefox driver
driver = webdriver.Chrome()

# Navigate to the URL
driver.get(url)

# Wait for the button to be clickable
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//button[@class="tooltip" and @tip="Convert the table below to comma-separated values\nsuitable for use with Excel"]'))
)

# Click the button
button.click()

# Close the browser
driver.quit()







### Manually Downloaded Team and Game Data from 2023 Season

In [ ]:
import pandas as pd

In [ ]:
games = pd.read_csv("23Games.csv")
standings = pd.read_csv("23Standings.csv")

In [ ]:
standings['Season'] = standings['Season'] % 10000
standings = standings.drop(columns = ['T', 'OT', 'FOW%'])
def replace_value(x):
    if isinstance(x, str) and x == "Montréal Canadiens":
        x = x.replace("Montréal Canadiens", "Montreal Canadiens")
        return x
    else:
        return x

standings = standings.applymap(replace_value)

### Prepare 2023 Data For Features to be Added

In [ ]:
games = games.drop(columns = ['Unnamed: 5', 'Att.', 'LOG', 'Notes'])
games['Winner'] = games.apply(lambda x: x['Visitor'] if x['G'] > x['G.1'] else (x['Home'] if x['G'] < x['G.1'] else 'Tie'), axis=1)
games = games[games['Winner'] != 'Tie']
games['v_elo'] = 1500
games['h_elo'] = 1500
games['v_winstreak'] = 0
games['h_winstreak'] = 0

### Calculate ELO of Teams in Games this Year

In [ ]:
def find_last_game_elo(df, index, team):
    for i in range(index -1, -1, -1):
        row = df.iloc[i]
        visitor_team = row['Visitor']
        home_team = row['Home']
        visitor_elo = row['v_elo']
        home_elo = row['h_elo']
        if visitor_team == team:
            return visitor_elo
        elif home_team == team:
            return home_elo
    return 1500

In [ ]:
def calculate_elo_game(df, index, game, visitor, home):
    visitor_elo = find_last_game_elo(df, index, visitor)
    home_elo = find_last_game_elo(df, index, home)
    mov_const = ((20 * (abs(game['G'] - game['G.1']) + 1.5)) ** .8) / (7.5 + (visitor_elo - home_elo) * 0.0001)

    elo_change = mov_const/100

    if game['Winner'] == game['Visitor']:
        visitor_elo += elo_change
        home_elo -= elo_change
    else:
        visitor_elo -= elo_change
        home_elo += elo_change

    return visitor_elo, home_elo

In [ ]:
def update_elo(df):
    for index, game in df.iterrows():
        visitor = game['Visitor']
        home = game['Home']

        visitor, home = calculate_elo_game(df, index, game, visitor, home)

        df.at[index, 'v_elo'] = visitor
        df.at[index, 'h_elo'] = home
    return df
update_elo(games)

### Calculate Winstreak of Teams in Games This Year

In [ ]:
def find_last_game(df, index, team):
    for i in range(index - 1, -1, -1):
        row = df.iloc[i]
        if row['Visitor'] == team or row['Home'] == team:
            return row
    return None

In [ ]:
def calculate_winstreak(df):
    for index, game in df.iterrows():
        h_game = find_last_game(df, index, game['Home'])
        v_game = find_last_game(df, index, game['Visitor'])
        if v_game is None:
            df.loc[index, "v_winstreak"] = 0
        else:
            df.loc[index, "v_winstreak"] = v_game['v_winstreak'] + 1 if v_game['Winner'] == game['Visitor'] else 0

        if h_game is None:
            df.loc[index, "h_winstreak"] = 0
        else:
            df.loc[index, "h_winstreak"] = h_game['h_winstreak'] + 1 if h_game['Winner'] == game['Home'] else 0

    return df
calculate_winstreak(games)

## Prediction Making!

In [ ]:
import numpy as np

### Create Feature Vector With Same Structure as the Model was Trained On

In [ ]:

import pandas as pd

def format_game_data(game, standing_df, game_df):
    # Extract relevant information from the game dataframe
    date, time, home_team, away_team = game
    home_row = standing_df.loc[standing_df['Team'] == home_team]
    away_row = standing_df.loc[standing_df['Team'] == away_team]

    # Find the most recent game for each team
    v_elo, v_winstreak = find_most_recent_game(game_df, away_team)
    h_elo, h_winstreak = find_most_recent_game(game_df, home_team)

    # Combine the relevant data into a single input vector
    elo_winstreak = np.array([v_elo, h_elo, v_winstreak, h_winstreak])
    input_vec = np.concatenate([away_row.values[0], home_row.values[0], elo_winstreak])

    # Convert the input vector to a dataframe with one row and 44 columns
    columns = list(away_row.columns) + list(home_row.columns) + ['away_elo', 'home_elo', 'away_winstreak', 'home_winstreak']
    df = pd.DataFrame(input_vec.reshape(1, -1), columns=columns)

    return df


def find_most_recent_game(df, team):
    for i in range(df.shape[0] - 1, -1, -1):
        row = df.iloc[i]
        if row['Visitor'] == team:
            return row['v_elo'], row['v_winstreak']
        elif row['Home'] == team:
            return row['h_elo'], row['h_winstreak']
    return None, None

In [ ]:
from tensorflow import keras

In [ ]:
model = keras.models.load_model("Best_Model")

### Make Predictions for the Day!!!

In [ ]:

# Create an empty list to store predictions
predictions = []

# Loop through each game in games_today
for game in games_today:

    date, time, home_team, away_team = game
    # Format the game data
    data = format_game_data(game, standings, games)
    # Make a prediction on the formatted data
    data = data.drop(columns = ['Team', 'Season'])

    prediction = model.predict(data.astype(int))

    print("There is a", prediction[0][0], "probability that the", home_team, "win!")


